# We import the libraries

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Functions

In [17]:
def read_data(filepath):
    """ Read the data from the file and return the distance matrix and the list of cities positions """

    with open(filepath, 'r') as file:
        distance_matrix = False
        city_list = False
        for line in file:
            line = line.strip()
            # Create the distance matrix and the list of cities
            if line.startswith('DIMENSION'):
                dimension = int(line.split()[-1])
                cities = np.zeros((dimension, 2))
                distances = np.zeros((dimension, dimension))
                i=0
                j=0
            
            # What do we do with the line
            if line.startswith('EDGE_WEIGHT_SECTION'):
                distance_matrix = True
                continue
            elif line.startswith('DISPLAY_DATA_SECTION'):
                distance_matrix = False
                city_list = True
                continue
            elif line.startswith('EOF'):
                break
            
            # Save data
            if distance_matrix:
                distances[i,] = [int(x) for x in line.split()]
                i += 1
            if city_list:
                cities[j,] = line.split()[-2:]
                j += 1
        
        return distances, cities

In [ ]:
class TSP_Genetic:
    """Perform a Genetic Algorithm simulation for the TSP problem"""

    def __init__(
        self,
        offspring_size,
        generations=100,
        print_rate=10,
        m_rate=0.05,
        c_rate=0.8,
        select_parents='tournament',
        crossover,
        mutate,
        select_survivors
    ):

        """Initialize the Genetic Algorithm simulation for the TSP problem.
            Args:
                offspring_size: The number of offspring to generate in each generation.
                generations: The number of generations to simulate in the genetic algorithm (100 by default).
                print_rate: How often to print the progress of the algorithm (Default: every 10 generations).
                m_rate: The mutation rate, representing the probability of mutation in offspring (Default: 0.05).
                c_rate: The crossover rate, representing the probability of crossover between parent chromosomes (Default: 0.8).
                select_parents: The method to select parents for crossover. Options are 'tournament' and 'roulette' (Default: 'tournament').
        """
        self.offspring_size = offspring_size
        self.generations = generations
        self.print_rate = print_rate

        self.m_rate = m_rate
        self.c_rate = c_rate

        self.select_parents = select_parents
        self.crossover = crossover
        self.mutate = mutate
        self.select_survivors = select_survivors


    def run(self,population,cities,distances):
        """Run the Genetic Algorithm simulation for the TSP problem.
            Args:
                population: The initial population of chromosomes.
                cities: The list of positions of each city. A row represents a city and the first element its x coordinate and the second its y coordinate.
                distances: A matrix representing the distances between each pair of cities.
        """
        self.population = population
        self.population_size = self.population.shape[0]
        self.cities = np.array(cities)
        self.n_cities = self.cities.shape[0]
        self.distances = np.array(distances)

        for generation in range(self.generations):
            # Select parents
            parents = self.select_parents()
            # Generate offspring
            offspring = self.crossover(parents)
            # Mutate offspring
            offspring = self.mutate(offspring)
            # Select survivors
            self.population = self.select_survivors(parents,offspring)
            # Print progress
            if generation % self.print_rate == 0:
                print(f"Generation {generation} - Best fitness: {self.fitness(self.population)[0]}")
        
        return self.population[0]

In [18]:
distances,cities= read_data('bays29.tsp')
    